In [1]:
import random
import time
import requests
import pandas as pd
import numpy as np

In [11]:
def compute_value(_hood, timestamp):
    """This would be replaced by a real algorithm that computed values 
    based on which hood, the environment, etc..."""
    return [random.random(), timestamp]

# the x-api-key is the secret authentication to allow posting updates
headers = {"x-api-key": "LfSpEZ3woOuljzb2EAg0"}
# the endpoint URL for trend updates
TREND_API_URL = "https://7zhs2wplt9.execute-api.us-east-1.amazonaws.com/dev/trend"
# time in seconds that the request will timeout if no response is received.
# the API will only run for 6 seconds, so if it can't process
# the data posted in that amount of time it will fail.
API_TIMEOUT = 6

# timestamps
now = int(time.time())
last_hour = now - now % 3600
start_time = last_hour - (3600 * 6)

# a list of points
points = [
    "Biotech.GroundFloor.Lab_g54.Hood_1.energy.occ",
    "Biotech.GroundFloor.Lab_g54.Hood_1.energy.unocc",
    "Biotech.GroundFloor.Lab_g54.Hood_1.sashOpenTime.occ",
]

# insert some data and log the results
for point in points:
    data = [compute_value(point, ts) for ts in range(start_time, last_hour, 3600)]
    json_obj = {"object_name": point, "data": data }
    # e.g.
    # { "object_name": "Biotech.GroundFloor.Lab_g54.Hood_1.energy.occ",
    # "data": [[0, 1681462800], [0, 1681466400], [0, 1681470000], [4.430475, 1681473600]],
    # }
    req_start = time.time()
    response = requests.post(TREND_API_URL, json=json_obj, headers=headers, timeout=API_TIMEOUT)
    req_end = time.time()
    r = response.json()
    print(f"""received status_code:{response.status_code} msg:{r.get('message', None)}
    rows:{r.get('affectedRows', None)} duplicates:{r.get('duplicates', None)} 
    for point:{point} with id:{r.get('obj_id', None)} in {req_end-req_start:.3f} seconds.""")

received status_code:200 msg:Insert
    rows:12 duplicates:0 
    for point:Biotech.GroundFloor.Lab_g54.Hood_1.energy.occ with id:11233324 in 0.182 seconds.
received status_code:200 msg:Insert
    rows:12 duplicates:6 
    for point:Biotech.GroundFloor.Lab_g54.Hood_1.energy.unocc with id:11233325 in 0.142 seconds.
received status_code:200 msg:Insert
    rows:12 duplicates:0 
    for point:Biotech.GroundFloor.Lab_g54.Hood_1.sashOpenTime.occ with id:11233326 in 0.120 seconds.


In [14]:
def create_tuple(response):
    response_data = response.json()
    response_datum = response_data[0]
    response_target = response_datum['target']
    response_datapoints = response_datum['datapoints']
    tuple_array = [tuple(x) for x in response_datapoints]
    npa = np.array(tuple_array, dtype=[
        ('value', np.double), ('ts', 'datetime64[ms]')])
    return npa

def synthetic_query(target, start, end):
    url = "https://portal.emcs.cornell.edu/api/datasources/proxy/5/query"
    data = {
        "range": {
          "from": "2023-04-25T00:00:00.000",
          "to": "2023-04-27T00:00:00.000",
        },
        "targets": [
          {
            "target": "Biotech.GroundFloor.Lab_g54.Hood_1.energy.occ"
          }
        ],

      }
    request = requests.post(url, json=data)
    print(request)
    # print(request.json())
    return create_tuple(request)

display(synthetic_query(target="Biotech.GroundFloor.Lab_g54.Hood_1.energy.occ", start="2023-04-25T00:00:00.000", end="2023-04-27T00:00:00.000"))
display(synthetic_query(target="Biotech.GroundFloor.Lab_g54.Hood_1.energy.unocc", start="2023-04-25T00:00:00.000", end="2023-04-27T00:00:00.000"))

<Response [200]>


array([(0.33125, '2023-04-26T07:00:00.000'),
       (0.91567, '2023-04-26T08:00:00.000'),
       (0.09101, '2023-04-26T09:00:00.000'),
       (0.85203, '2023-04-26T10:00:00.000'),
       (0.29085, '2023-04-26T11:00:00.000'),
       (0.95329, '2023-04-26T12:00:00.000')],
      dtype=[('value', '<f8'), ('ts', '<M8[ms]')])

<Response [200]>


array([(0.33125, '2023-04-26T07:00:00.000'),
       (0.91567, '2023-04-26T08:00:00.000'),
       (0.09101, '2023-04-26T09:00:00.000'),
       (0.85203, '2023-04-26T10:00:00.000'),
       (0.29085, '2023-04-26T11:00:00.000'),
       (0.95329, '2023-04-26T12:00:00.000')],
      dtype=[('value', '<f8'), ('ts', '<M8[ms]')])